In [98]:
import sys
import os
import cairosvg
from IPython.core.display import display, Markdown
from PIL import Image
from json_dict import JsonDict
sys.path.append(os.path.abspath(".."))
import main
import pandas as pd
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib
import matplotlib.pyplot as plt
import pickle
os.chdir(os.path.dirname(main.__file__))

In [99]:
FORCE_NEW=False
SHOW_IMAGES=False
SEED=1995
DPI=300
#EVALUATION_PARAMETER
c_min=0
c_max=30
deg_min=10
deg_max=300

pixel=100*100

true_name="z_average"
predicted_name="predicted_z_average"
metric_steps=["training","validation"]
metric_epoch_limit=2000
metric_highlighted_epochs=[10,50,99,500]

display_quantiles = [1.,0.99,0.95,0.90]

random_comparison_true_pred_number=40

In [100]:
#PRECALCULAIONS
pixerl_1d=np.sqrt(pixel)
c_delt=(c_max-c_min)/pixerl_1d
deg_delt=(deg_max-deg_min)/(pixerl_1d-1)

c_delt =(c_max-c_min)/(pixerl_1d-1)

concentrations = np.arange(0,30,0.1)
poly_deg = np.arange(deg_min,deg_max+deg_delt,deg_delt)
concentrations = np.arange(c_min,c_max+c_delt,c_delt)

In [113]:
%matplotlib inline

#FUNCTIONS
def load_config(model_path):
    if not os.path.exists(os.path.join(model_path,"config.json")):
        raise FileNotFoundError("Model config '{}' not found".format(os.path.join(model_path,"config.json")))
    return JsonDict(os.path.join(model_path,"config.json"))

def load_model(config):
    model = main.load_model(config.getsubdict("model"))
    return model

def load_data(config):
    training_history=config.get("training","training_history",default=[])
    if len(training_history)<1:
        raise ValueError("model '{}' has no training history".format(config.get("model","name")))
    training_data = training_history[-1]['args']['train']
    if not os.path.exists(training_data):
        raise FileNotFoundError("Training data '{}' not found".format(training_data))

    train_df = pd.read_csv(training_data)

    test_data = training_data.replace("train_data_size_kept","train_data_size_removed")
    if not os.path.exists(test_data):
            raise FileNotFoundError("Training data '{}' not found".format(test_data))
    test_df = pd.read_csv(test_data)
    return train_df,test_df

def data_prediction(model,train_df,test_df,eval_dir):
    def _pred(df):
        if np.any(np.isnan(df["dp"])):
            raise ValueError("DP is None in for data with '{}'".format(model.name) )
        df =  main.predict(model,df)
        df["error"] = np.abs(df[predicted_name]-df[true_name])
        df["rel_error"] = df["error"]/df[true_name]
        return df

    new=False
    n_train_df=None
    if os.path.exists(os.path.join(eval_dir,"train_df.csv")):
        n_train_df = pd.read_csv(os.path.join(eval_dir,"train_df.csv"))

    if n_train_df is None or n_train_df.shape[0] != train_df.shape[0] or \
                not n_train_df[['circular_smiles_10']].equals(train_df[['circular_smiles_10']]) or \
                not np.allclose(n_train_df[['dp','polymer_concentration',"pdi","with_pva","z_average"]].values,
                                                train_df[['dp','polymer_concentration',"pdi","with_pva","z_average"]].values
                                            ) or \
                 not np.allclose(main.predict(model,train_df.iloc[[0]],verbose=False)[predicted_name].values,
                                 n_train_df.iloc[[0]][predicted_name].values
                                 ):
        n_train_df= _pred(train_df)
        n_train_df.to_csv(os.path.join(eval_dir,"train_df.csv"),index=None)
        new=True

    n_test_df=None
    if os.path.exists(os.path.join(eval_dir,"test_df.csv")):
        n_test_df = pd.read_csv(os.path.join(eval_dir,"test_df.csv"))

    if n_test_df is None or n_test_df.shape[0] != test_df.shape[0] or \
                not n_test_df[['circular_smiles_10']].equals(test_df[['circular_smiles_10']]) or \
                not np.allclose(n_test_df[['dp','polymer_concentration',"pdi","with_pva","z_average"]].values,
                                                test_df[['dp','polymer_concentration',"pdi","with_pva","z_average"]].values
                                            ) or \
            not np.allclose(main.predict(model,test_df.iloc[[0]],verbose=False)[predicted_name].values,
                            n_test_df.iloc[[0]][predicted_name].values
                            ):
        n_test_df= _pred(test_df)
        n_test_df.to_csv(os.path.join(eval_dir,"test_df.csv"),index=None)
        new=True

    return n_train_df,n_test_df,new

def display_metrics(model,eval_dir):
    td = pickle.load(open(os.path.join(model.get_dir(),"training","training_metrics.mtr"), "rb"))
    steps=[]
    epochs=[]
    metrics=[]
    for step, data in td.items():
        if step == "epochs":
            continue
        steps.append(step)
        for epoch, dmetrics in data.items():
            if epoch not in epochs:
                epochs.append(epoch)
            for metric,mdata in dmetrics.items():
                if metric not in metrics:
                    metrics.append(metric)
    epochs=np.sort(epochs)
    steps=[s for s in steps if s in metric_steps]

    data=np.zeros((len(steps),len(metrics),len(epochs)))*np.nan
    for i,step in enumerate(steps):
                for j,metric in enumerate(metrics):
                    for k,epoch in enumerate(epochs):
                        try:
                            data[i,j,k] = td[step][epoch][metric]
                        except KeyError as ke:
                            raise ke


    def _disp(from_epoch,to_epoch,name,highlights=[]):
        b=iter(["-","--",'-.',':'])
        plt.figure(dpi=300,figsize=(12,3))

        for i,step in enumerate(steps):
            for j,metric in enumerate(metrics):
                plt.plot(epochs[from_epoch:to_epoch],data[i,j,from_epoch:to_epoch],next(b),label=step.capitalize()+" "+metric)

        for h in highlights:
            if h>=from_epoch and h<=to_epoch:
                plt.axvline(x=h, lw=3, alpha=0.5)

        plt.xlabel("Epoch")
        plt.title("Loss")
        plt.ylabel("Loss value")
        plt.legend()

        plt.savefig(os.path.join(eval_dir,name+".png"),dpi=DPI)
        if SHOW_IMAGES:
            plt.show()
        plt.close()

    # w/o highlights
    # full epoch
    _disp( from_epoch = 0,to_epoch = len(epochs),name="full_training_metrics")
    # to fixed limit epoch
    _disp( from_epoch = 0,to_epoch = min(metric_epoch_limit,len(epochs)),name="fixed_training_metrics")

    #w highlights
    # full epoch
    _disp( from_epoch = 0,to_epoch = len(epochs),highlights=metric_highlighted_epochs,name="full_training_metrics_hl")
    # to fixed limit epoch
    _disp( from_epoch = 0,to_epoch = min(metric_epoch_limit,len(epochs)),highlights=metric_highlighted_epochs,name="fixed_training_metrics_hl")

def calc_metrics( train_df,test_df):
    comb_df=pd.concat([train_df,test_df])
    d={
              "training_relmae":np.abs(train_df["rel_error"]).mean(),
              "training_rmse":np.sqrt((train_df["error"]**2).mean()),
              "test_relmae":np.abs(test_df["rel_error"]).mean(),
              "test_rmse":np.sqrt((test_df["error"]**2).mean()),
                "combined_relmae":np.abs(comb_df["rel_error"]).mean(),
                "combined_rmse":np.sqrt((comb_df["error"]**2).mean()),
          }

    s="Metrics:<br>"
    s+="--training data:<br>"
    s+="----RELMAE: {}<br>".format(d["training_relmae"])
    s+="----RMSE: {}<br>".format(d["training_rmse"])
    s+="--test data:<br>"
    s+="----RELMAE: {}<br>".format(d["test_relmae"])
    s+="----RMSE: {}<br>".format(d["test_rmse"])
    s+="--combined data:<br>"
    s+="----RELMAE: {}<br>".format(d["combined_relmae"])
    s+="----RMSE: {}<br>".format(d["combined_rmse"])
    display(Markdown(s))
    return d


def highlighted_training_steps(model,highlighted,eval_dir):
    import json
    with open(os.path.join(model.get_dir(),"training","ValidationPointRecorderCallback.lst"), "r") as f:
        dlist=json.loads(f.read())
    available_epochs=[]
    for e in range(len(dlist)):
        if len(dlist[e][0]) > 0:
            available_epochs.append(e)
    for h in highlighted:
        e = available_epochs[np.argmin(np.abs(np.array(available_epochs)-h))]
        true,pred = dlist[e]
        true,pred = np.array(true),np.array(pred)
        _min = true.min()
        _max =true.max()
        over=0.05
        _min,_max = _min - over*(_max-_min) , _max + over*(_max-_min)

        plt.figure(dpi=100)
        plt.plot([_min,_max],[_min,_max],"--",alpha=0.5)

        plt.plot(true,pred,"bo",alpha=0.7,
                 markeredgecolor=(1,1,1),markersize=10,label="epoch {}".format(e))
        plt.xlabel("True")
        plt.ylabel("Predicted")
        plt.legend()

        plt.savefig(os.path.join(eval_dir,"highlighted_training_steps_{}.png".format(h)),dpi=DPI)
        if SHOW_IMAGES:
           plt.show()
        plt.close()

def get_mols(train_df,test_df,eval_dir):
    unique_s=list(train_df["circular_smiles_10"].unique())
    all_smiles = unique_s

    train_mols=Chem.Draw.MolsToGridImage([Chem.MolFromSmiles(smiles) for smiles in unique_s],molsPerRow=min(len(unique_s),5),subImgSize=(DPI,DPI))
    train_mols.save(os.path.join(eval_dir,"mols_train.png"))
    if SHOW_IMAGES:
        display(train_mols)

    unique_s = list(test_df["circular_smiles_10"].unique())
    all_smiles.extend(unique_s)

    test_mols = Chem.Draw.MolsToGridImage([Chem.MolFromSmiles(smiles) for smiles in unique_s],molsPerRow=min(len(unique_s),5),subImgSize=(DPI,DPI))
    test_mols.save(os.path.join(eval_dir,"mols_test.png"))
    if SHOW_IMAGES:
            display(test_mols)

    return {smiles:Chem.MolFromSmiles(smiles) for smiles in all_smiles}

def true_vs_predicted_by_mol(train_df,test_df,eval_dir):
    _min = min(train_df[true_name].min(),test_df[true_name].min())
    _max = max(train_df[true_name].max(),test_df[true_name].max())
    over=0.05
    _min,_max = _min - over*(_max-_min) , _max + over*(_max-_min)

    plt.figure(figsize=(12,6))
    plt.plot([_min,_max],[_min,_max],"--")
    for su in train_df.smiles_ru.unique():
        sd = train_df[train_df.smiles_ru == su]
        plt.plot(sd[true_name],sd[predicted_name],"o",label=su,alpha=0.7,
                 markeredgecolor="w",
                 markersize=5
                )
    plt.xlabel("True Z-Average [nm]")
    plt.ylabel("Predicted Z-Average [nm]")
    plt.legend()
    plt.savefig(os.path.join(eval_dir,"true_vs_false_mol_train.png"),dpi=DPI)
    if SHOW_IMAGES:
        plt.show()
    plt.close()

    plt.figure(figsize=(12,6))
    plt.plot([_min,_max],[_min,_max],"--")
    for su in test_df.smiles_ru.unique():
        sd = test_df[test_df.smiles_ru == su]
        plt.plot(sd[true_name],sd[predicted_name],"o",label=su,alpha=0.7,
                 markeredgecolor="b",
                 markersize=5
                )
    plt.xlabel("True Z-Average [nm]")
    plt.ylabel("Predicted Z-Average [nm]")
    plt.legend()
    plt.savefig(os.path.join(eval_dir,"true_vs_false_mol_test.png"),dpi=DPI)
    if SHOW_IMAGES:
        plt.show()
    plt.close()

    plt.figure(figsize=(12,6))
    plt.plot([_min,_max],[_min,_max],"--")
    for su in train_df.smiles_ru.unique():
        sd = train_df[train_df.smiles_ru == su]
        plt.plot(sd[true_name],sd[predicted_name],"o",label=su,alpha=0.7,
                 markeredgecolor="w",
                 markersize=5
                )
    for su in test_df.smiles_ru.unique():
            sd = test_df[test_df.smiles_ru == su]
            plt.plot(sd[true_name],sd[predicted_name],"o",label=su,alpha=0.7,
                     markeredgecolor="b",
                     markersize=5
                    )
    plt.xlabel("True Z-Average [nm]")
    plt.ylabel("Predicted Z-Average [nm]")
    plt.legend()
    plt.savefig(os.path.join(eval_dir,"true_vs_false_mol_both.png"),dpi=DPI)
    if SHOW_IMAGES:
        plt.show()
    plt.close()

def true_vs_predicted_quantilized(train_df,test_df,eval_dir):
    _min = min(train_df[true_name].min(),test_df[true_name].min())
    _max = max(train_df[true_name].max(),test_df[true_name].max())
    over=0.05
    _min,_max = _min - over*(_max-_min) , _max + over*(_max-_min)
    qs=list(sorted(display_quantiles))

    markercolors=["w","k","r","b"]

    def _plot(df,name):
        plt.figure(figsize=(12,6))
        plt.plot([_min,_max],[_min,_max],"--")
        dfs=[]
        if isinstance(df,(list,tuple)):
            assert isinstance(name,(list,tuple)),"name not a list or tuble but df is"
            if len(df)>1:
                df,dfs = df[0],df[1:]
                name,names=name[0],name[1:]
            else:
                df,dfs=df[0],[]
                name,names=name[0],[]

        sorted_indices = np.argsort(df["error"])



        for i,q in enumerate(qs):
                if i==0:
                    s=0
                else:
                    s=int(len(sorted_indices)*qs[i-1])
                e=int(q*len(sorted_indices))
                plt.plot(df[true_name].iloc[sorted_indices[s:e]],
                         df[predicted_name].iloc[sorted_indices[s:e]],
                         "o",
                         label=r"{} quantile ($\overline{{error}}={:.1f}\pm{:.1f}$%)".format(q,
                                                                                                  100*(df["rel_error"]).iloc[sorted_indices[:e]].mean(),
                                                                                                 100*(df["rel_error"]).iloc[sorted_indices[:e]].std()
                                                                                                 ),
                         alpha=0.7,
                         markeredgecolor=markercolors[0],
                         markersize=4/q**2,
                        )
        for i,sdf in enumerate(dfs):
            plt.plot(sdf[true_name],
                         sdf[predicted_name],
                         "o",
                         label=r"{} data ($\overline{{error}}={:.1f}\pm{:.1f}$%)".format(names[i],(100*sdf["rel_error"]).mean(),
                                                                                        100*(sdf["rel_error"]).std()
                                                                                        ),
                         alpha=0.7,
                         markeredgecolor=markercolors[i+1],
                         markersize=4
                        )
        plt.xlabel("True Z-Average [nm]")
        plt.ylabel("Predicted Z-Average [nm]")
        plt.legend()
        plt.savefig(os.path.join(eval_dir,"true_vs_predicted_quantilized_{}.png".format(name)),dpi=DPI)
        if SHOW_IMAGES:
            plt.show()
        plt.close()

    _plot(train_df,name="train")
    _plot(test_df,name="test")
    _plot(pd.concat([train_df,test_df]),name="both")
    _plot([train_df,test_df],name=["train_super_test","test"])

def random_comparison_true_pred(train_df,test_df,eval_dir):
    def _plot(df,name):
        true=df[true_name].values
        pred=df[predicted_name].values

        idx  = np.arange(len(true))
        np.random.seed(SEED)
        idx=np.random.choice(idx,size=random_comparison_true_pred_number)
        width=random_comparison_true_pred_number/100
        prehw=plt.rcParams['hatch.linewidth']
        plt.rcParams['hatch.linewidth']=width

        fig,a = plt.subplots()
        a.bar(np.arange(len(idx)) - width/2,true[idx],width,label="true", hatch="xxxxxx")
        a.bar(np.arange(len(idx)) + width/2,pred[idx],width,label="predicted", hatch="....")
        a.bar(np.arange(len(idx)),np.abs(true[idx]-pred[idx]),label="difference")

        plt.ylabel("Z-Average [nm]")
        plt.title("Random comparison of Z-Average predicted and true ")
        a.axes.xaxis.set_visible(False)
        plt.legend()
        plt.tight_layout()

        plt.savefig(os.path.join(eval_dir,"random_comparison_true_pred_{}.png".format(name)),dpi=DPI)
        if SHOW_IMAGES:
            plt.show()
        plt.close()


        fig,a = plt.subplots()
        a.bar(np.arange(len(idx)) - width/2,1,width,label="true", hatch="xxxxxx")
        a.bar(np.arange(len(idx)) + width/2,1*pred[idx]/true[idx], width,label="predicted", hatch="....")
        a.bar(np.arange(len(idx)), np.abs(1 - (1*pred[idx]/true[idx])) ,label="difference")

        plt.ylabel("rel. Z-Average ")
        plt.title("Random comparison of Z-Average predicted and true ")
        a.axes.xaxis.set_visible(False)
        plt.legend()
        plt.tight_layout()

        plt.savefig(os.path.join(eval_dir,"random_comparison_true_pred_{}_rel.png".format(name)),dpi=DPI)
        if SHOW_IMAGES:
            plt.show()
        plt.close()


        plt.rcParams['hatch.linewidth']=prehw

    _plot(train_df,name="train")
    _plot(test_df,name="test")
    _plot(pd.concat([train_df,test_df]),name="both")

def calculate_prediction_grid(model,all_mols,eval_dir):
    #full_df=pd.DataFrame(columns=['dp','polymer_concentration',"pdi","with_pva",'circular_smiles_10'])
    df_data=[]
    for smiles in all_mols.keys():
        for conc in concentrations:
            for deg in poly_deg:
                for wpva in [0,1]:
                    df_data.append({
                        'dp' : deg,
                        'polymer_concentration' : conc,
                        "pdi":1,
                        "with_pva":wpva,
                        'circular_smiles_10':smiles
                    })

    pred_df=None
    new=False
    if os.path.exists(os.path.join(eval_dir,"prediction_grid.csv")):
        pred_df=pd.read_csv(os.path.join(eval_dir,"prediction_grid.csv"))
    full_df = pd.DataFrame(df_data)

    if pred_df is None or pred_df.shape[0] != full_df.shape[0] or \
            not pred_df[['circular_smiles_10']].equals(full_df[['circular_smiles_10']]) or \
            not np.allclose(pred_df[['dp','polymer_concentration',"pdi","with_pva"]].values,
                                            full_df[['dp','polymer_concentration',"pdi","with_pva"]].values
                                        ) or \
             not np.allclose(main.predict(model,full_df.iloc[[0]],verbose=False)[predicted_name].values,
                             pred_df.iloc[[0]][predicted_name].values
                             ):
        pred_df = main.predict(model,full_df)
        pred_df.to_csv(os.path.join(eval_dir,"prediction_grid.csv"),index=None)
        new=True
    return pred_df,new

def plot_prediction_grid(prediction_df,train_df,test_df,eval_dir,max_h=None,min_h=0):
    if max_h is None:
        max_h=max(train_df[true_name].max(),test_df[true_name].max())
    z_max=max_h
    z_min=min_h

    #import cairosvg
    #from PIL import Image
    #import os
    def to_rdk_transparent_png(mol ):
        dr = Chem.Draw.rdMolDraw2D.MolDraw2DSVG( DPI, DPI )
        #dr.SetFontSize( 0.27 )
        op = dr.drawOptions()
        #for i in range( mol.GetNumAtoms() ) :
        #    op.atomLabels[i]=mol.GetAtomWithIdx(i).GetSymbol()+str((i+1))
        Chem.AllChem.Compute2DCoords( mol )
        dr.DrawMolecule( mol )
        dr.FinishDrawing()

        svg = dr.GetDrawingText()
        svg =  svg.replace("<rect style='opacity:1.0;fill:#","<rect style='opacity:0.0;fill:#")

        cairosvg.svg2png(svg,write_to ="temp.png",dpi=DPI)
        img = Image.open("temp.png")
        return img

    for (smiles,with_pva), smiles_df in prediction_df.groupby(["circular_smiles_10","with_pva"]):
        sd = train_df[(train_df["circular_smiles_10"]==smiles)]
        if len(sd.dp.unique()) >0:
            pass
        else:
            sd = test_df[(test_df["circular_smiles_10"]==smiles)]

        plt.figure(dpi=150)
        predicted_z_average = smiles_df["predicted_z_average"].values.astype(float)
        #print(predicted_z_average)
        XY_pred = (predicted_z_average.reshape(len(concentrations),-1)).T
        plt.imshow(XY_pred,
                        #extent=[c_min,c_max,30,0],
             #            yticklabels=yticks
                   cmap='rocket',
                        )
        plt.clim(z_min,z_max)


        sd = sd[sd["with_pva"]==with_pva]
        #sd = sd[sd["with_pva"]==with_pva]
        dps=[]
        poly_conc=[]
        z_average=[]
        for (dp,polymer_concentration), point_group in sd.groupby(["dp","polymer_concentration"]):
            dps.append(dp)
            poly_conc.append(polymer_concentration)
            z_average.append(point_group["z_average"].mean())
        new_dps=[np.argmin(np.abs(poly_deg-deg)) for deg in dps]
        new_conc=[np.argmin(np.abs(concentrations-conc)) for conc in poly_conc]

        k=7
        old_xticks = plt.xticks()
        plt.xticks(np.linspace(0,old_xticks[0].max(),k), np.round(np.linspace(concentrations.min(),concentrations.max(),k)))

        old_yticks = plt.yticks()
        plt.yticks(np.linspace(0,len(poly_deg),k), np.linspace(poly_deg.min(),poly_deg.max(),k,dtype=int))

        plt.scatter(new_conc,new_dps,c=z_average,edgecolors="w")
        plt.clim(z_min,z_max)

        plt.xlabel("Polymer concentration [g/L]")
        plt.ylabel("Degree of polymerization")

        title="{} {} PVA".format(sd.polymer.unique()[0], "with" if with_pva else "without",)
        plt.title(title)

        trans_mol=to_rdk_transparent_png(Chem.MolFromSmiles(smiles))
        if SHOW_IMAGES:
            display(trans_mol)
        trans_mol.save(os.path.join(
            eval_dir,
            "trans_mol_{}.png".format(sd.polymer.unique()[0]).replace(" ","_")
        ))
        cbar = plt.colorbar()

        cbar.ax.get_yaxis().labelpad = 15
        cbar.ax.set_ylabel('Z-Average [nm]', rotation=270)

        plt.savefig(
            os.path.join(
                eval_dir,"prediction_grid_{}.png".format(title.replace(" ","_"))
            ),dpi=DPI
        )
        if SHOW_IMAGES:
            plt.show()

        plt.close()

        #xypred = prednp.reshape((d_res,c_res))

def plot_prediction_grid_3d(prediction_df,train_df,test_df,eval_dir,max_h=None,min_h=0):
    if max_h is None:
            max_h=max(train_df[true_name].max(),test_df[true_name].max())
    z_max=max_h
    z_min=min_h

    for (smiles,with_pva), smiles_df in prediction_df.groupby(["circular_smiles_10","with_pva"]):
        sd = train_df[(train_df["circular_smiles_10"]==smiles)]
        if len(sd.dp.unique()) >0:
            pass
        else:
            sd = test_df[(test_df["circular_smiles_10"]==smiles)]
        title="{} {} PVA".format(sd.polymer.unique()[0], "with" if with_pva else "without",)

        fig = plt.figure(dpi=300)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(elev=15., azim=210)
        predicted_z_average = smiles_df["predicted_z_average"].values.astype(float)
        Z = (predicted_z_average.reshape(len(concentrations),-1)).T

        norm=matplotlib.colors.Normalize(vmin=z_min, vmax=z_max)
        X, Y = np.meshgrid(concentrations, poly_deg)
        p = ax.plot_surface(X,Y,Z,
                          linewidth=0.1,
                        cmap='rocket',
                            norm=norm
                         )

        ax.set_zlim(z_min,z_max)

        m = cm.ScalarMappable(cmap=p.cmap,norm=norm)
        #m.set_array(np.array([z_min,z_max]))
        #m.to_rgba()
        plt.colorbar(m)

    #    cbar = fig.colorbar(p)
    #    cbar.set_clim(z_min,z_max)


        plt.savefig(
                    os.path.join(
                        eval_dir,"prediction_grid_3d_{}.png".format(title.replace(" ","_"))
                    ),dpi=DPI
                )
        if SHOW_IMAGES:
            plt.show()
        plt.close()


        #xypred = prednp.reshape((d_res,c_res))

def merge_images(eval_dir):
    top="fixed_training_metrics.png"
    second="true_vs_false_mol_both.png"
    third="true_vs_predicted_quantilized_train_super_test.png"
    bottom_left="random_comparison_true_pred_both_rel.png"
    bottom_right="random_comparison_true_pred_both.png"

    GRIDS_PER_ROW=4

    top,second,third,bottom_left,bottom_right = (os.path.join(eval_dir,n) for n in [top,second,third,bottom_left,bottom_right])
    top,second,third,bottom_left,bottom_right = (Image.open(n) for n in [top,second,third,bottom_left,bottom_right])
    dist=150

    full_height=np.sum(
        [s.size[1]+dist for s in  [top,second,third]] +
        [np.max([s.size[1]+dist for s in  [bottom_left,bottom_right]])]
    )

    full_width=np.max([s.size[0]+dist for s in  [top,second,third]]+
                      [np.sum([s.size[0]+dist for s in  [bottom_left,bottom_right]])]
                      )

    new_img = Image.new('RGBA',(full_width,full_height))
    x_offset=0
    y_offset=0

    new_img.paste(top,(x_offset,y_offset))
    y_offset+=top.size[1]+dist

    new_img.paste(second,(x_offset,y_offset))
    y_offset+=second.size[1]+dist

    new_img.paste(third,(x_offset,y_offset))
    y_offset+=third.size[1]+dist

    new_img.paste(bottom_left,(x_offset,y_offset))
    x_offset+=bottom_left.size[0]+dist

    new_img.paste(bottom_right,(x_offset,y_offset))

    new_img.save(os.path.join(eval_dir,"merge_images_1.png"))

    files = [os.path.join(eval_dir,"prediction_grid",s) for s in sorted(os.listdir(os.path.join(eval_dir,"prediction_grid"))) if s.startswith("prediction_grid")]

    size=np.array(Image.open(files[0]).size)+dist

    line=0
    row=0
    new_img = Image.new('RGBA',(int(size[0]*GRIDS_PER_ROW),int(size[1]*np.ceil(len(files)/GRIDS_PER_ROW))))
    for i,f in enumerate(files) :
        new_img.paste(Image.open(f),(int(row*size[0]),int(line*size[1])))
        row += 1

        if row >= GRIDS_PER_ROW:
            line +=1
            row=0

    new_img.save(os.path.join(eval_dir,"merge_images_2.png"))

def crop_images(eval_dir):
    for folder in [os.path.join(eval_dir,s) for s in os.listdir(eval_dir) if os.path.isdir(os.path.join(eval_dir,s))]:
        crop_images(folder)

    files = [os.path.join(eval_dir,s) for s in os.listdir(eval_dir) if s.endswith(".png")]
    for file in files:
        image=Image.open(file)
        image.load()

        image_data = np.asarray(image)
        if image_data.shape[2]<4:
            continue

        alpha=image_data[:,:,3]
        non_empty_columns = np.where(alpha.max(axis=0)>0)[0]
        non_empty_rows = np.where(alpha.max(axis=1)>0)[0]

        cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))

        image_data_new = image_data[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1 , :]

        new_image = Image.fromarray(image_data_new)
        new_image.save(file)

def evaluate(model_path):
    config=load_config(model_path)
    model = load_model(config)
    eval_dir = os.path.join(model_path,"evaluation")
    os.makedirs(eval_dir,exist_ok=True)
    display(Markdown("Evaluate {}:".format(model.name)))
    display_metrics(model,eval_dir=eval_dir)
    train_df,test_df = load_data(config)
    train_df,test_df,new = data_prediction(model,train_df,test_df,eval_dir)

    all_mols = get_mols(train_df,test_df,eval_dir)
    metrics = calc_metrics( train_df,test_df)
    if new or FORCE_NEW:
        highlighted_training_steps(model,metric_highlighted_epochs,eval_dir=eval_dir)
        true_vs_predicted_by_mol(train_df,test_df,eval_dir)
        true_vs_predicted_quantilized(train_df,test_df,eval_dir)
        random_comparison_true_pred(train_df,test_df,eval_dir)

    prediction_grid,new2=calculate_prediction_grid(model,all_mols,eval_dir)

    if new or new2 or FORCE_NEW:
        os.makedirs(os.path.join(eval_dir,"prediction_grid"),exist_ok=True)
        plot_prediction_grid(prediction_grid,train_df,test_df,os.path.join(eval_dir,"prediction_grid"))
        os.makedirs(os.path.join(eval_dir,"prediction_grid_3d"),exist_ok=True)
        plot_prediction_grid_3d(prediction_grid,train_df,test_df,os.path.join(eval_dir,"prediction_grid_3d"))

        crop_images(eval_dir)
        merge_images(eval_dir)

    display(Markdown("<br>"))
    return {**{"path":model_path,"name":model.name},**metrics,**config['model']}


In [114]:
source_folder="pretrained/"
print(sorted(os.listdir(source_folder),key=lambda f: int(f.rsplit("_",maxsplit=1)[1])))
eval_data=[]
for model_path in sorted(os.listdir(source_folder),
                         #reverse=True,
                         key=lambda f: int(f.rsplit("_",maxsplit=1)[1])
                         ):
    ev = evaluate(os.path.join(source_folder,model_path))

    eval_data.append(ev)
    break
eval_df=pd.DataFrame(eval_data)
display(eval_df)

['np_model_0', 'np_model_1', 'np_model_2', 'np_model_3', 'np_model_4', 'np_model_5', 'np_model_6', 'np_model_7', 'np_model_8', 'np_model_9', 'np_model_10', 'np_model_11', 'np_model_12', 'np_model_13']
try loading model from /home/julian/IdeaProjects/nanoparticle_size_prediction/pretrained/np_model_0/np_model_0.pth
load successful
load data 2276/2276 (100.00%)    
load data 531/531 (100.00%)    
load data 200000/200000 (100.00%)    


Evaluate np_model_0:

Metrics:<br>--training data:<br>----RELMAE: 0.04614023494936432<br>----RMSE: 18.455068518187538<br>--test data:<br>----RELMAE: 2.8103035566802834<br>----RMSE: 936.2270936024782<br>--combined data:<br>----RELMAE: 0.5690368234207281<br>----RMSE: 407.53847769332174<br>

<br>

,path,name,training_relmae,training_rmse,test_relmae,test_rmse,combined_relmae,combined_rmse,additional_input_names,backend,fcn_model,featurizer,gcn_layer_sizes,gcn_output_dims,learning_rate,loss_function,metrics,pooling,smiles_column,task_names
0,pretrained/np_model_0,np_model_0,0.04614,18.455069,2.810304,936.227094,0.569037,407.538478,"[dp, with_pva, polymer_concentration]",pytorch_geometric,"{'layer': [{'args': [67, 32], 'module': 'Linea...","[atom_symbol_hcnopsclbr_other_one_hot, atom_de...","[None, None, None, None, None, None]",64,0.001,"{'kwargs': {}, 'name': 'RMAE'}","[relmae, rmse]","[weight_sum, max]",circular_smiles_10,[z_average]


In [ ]:
#model.save_last_prediction_tensor=True
#model.save_last_input_data=True

#test_df.loc[np.isnan(test_df["dp"]),"dp"]=400
#if "level_0" in test_df.columns:
#    test_df.drop("level_0",axis=1,inplace=True)




#mask = test_df["dp"]== test_df["dp"]-1
#n_mask = test_df["dp"] == (test_df[test_df["rel_error"] == test_df["rel_error"].max()]["dp"]).unique()[0]
#mask = n_mask

#n_mask = test_df["dp"] == (test_df[test_df["rel_error"] == test_df[~mask]["rel_error"].max()]["dp"]).unique()[0]
#mask = mask | n_mask 

#raise Exception("remove me")

#python main.py --train="__last" --model_path="pretrained/np_model_11"

[100]